In [74]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, pipeline, Trainer, DataCollatorWithPadding, TrainingArguments, AutoModelForQuestionAnswering, AutoTokenizer
from datasets import Dataset,load_dataset
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
import requests
import torch
import numpy as np

In [75]:
bert_tokenizer=BertTokenizerFast.from_pretrained('bert-large-uncased',return_token_type_ids=True)
qa_bert=BertForQuestionAnswering.from_pretrained('bert-large-uncased')
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
qa_bert.to(device)
print(device)

loading file vocab.txt from cache at /Users/vishalsankarram/.cache/huggingface/hub/models--bert-large-uncased/snapshots/6da4b6a26a1877e173fca3225479512db81a5e5b/vocab.txt
loading file tokenizer.json from cache at /Users/vishalsankarram/.cache/huggingface/hub/models--bert-large-uncased/snapshots/6da4b6a26a1877e173fca3225479512db81a5e5b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/vishalsankarram/.cache/huggingface/hub/models--bert-large-uncased/snapshots/6da4b6a26a1877e173fca3225479512db81a5e5b/tokenizer_config.json
loading configuration file config.json from cache at /Users/vishalsankarram/.cache/huggingface/hub/models--bert-large-uncased/snapshots/6da4b6a26a1877e173fca3225479512db81a5e5b/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "c

mps


In [76]:
squad = load_dataset("squad_v2")

Reusing dataset squad_v2 (/Users/vishalsankarram/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 69.77it/s]


In [77]:
print(squad['train'][0])
print(squad['train'][0]['answers'])

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}
{'text': ['in the late 1990s'], 'answer_start': [269]}


In [78]:
def preprocess_data(examples):
    questions = examples["question"]
    contexts = examples["context"]
    answers = examples["answers"]

    # Tokenize inputs (truncating and padding to a max length)
    inputs = bert_tokenizer(questions, contexts, truncation=True, padding="max_length", max_length=384)

    # Find start and end positions of the answer
    start_positions = []
    end_positions = []

    for i, answer in enumerate(answers):
        if len(answer["text"]) == 0:  # Unanswerable
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_pos = inputs.char_to_token(i, answer["answer_start"][0])
            end_pos = inputs.char_to_token(i, answer["answer_start"][0] + len(answer["text"][0]) - 1)

            # Handle special cases when answer is out of bounds
            start_positions.append(start_pos if start_pos is not None else 0)
            end_positions.append(end_pos if end_pos is not None else 0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_squad = squad.map(preprocess_data, batched=True)

Loading cached processed dataset at /Users/vishalsankarram/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-435f0d6d59b212fc.arrow
Loading cached processed dataset at /Users/vishalsankarram/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-a689eec5f612dbc2.arrow


In [79]:
tokenized_squad["train"] = tokenized_squad["train"].remove_columns(["id", "title", "context", "question", "answers"])
tokenized_squad["validation"] = tokenized_squad["validation"].remove_columns(["id", "title", "context", "question", "answers"])

In [80]:
tokenized_squad["train"][0]

{'input_ids': [101,
  2043,
  2106,
  20773,
  2707,
  3352,
  2759,
  1029,
  102,
  20773,
  21025,
  19358,
  22815,
  1011,
  5708,
  1006,
  1013,
  12170,
  23432,
  29715,
  3501,
  29678,
  12325,
  29685,
  1013,
  10506,
  1011,
  10930,
  2078,
  1011,
  2360,
  1007,
  1006,
  2141,
  2244,
  1018,
  1010,
  3261,
  1007,
  2003,
  2019,
  2137,
  3220,
  1010,
  6009,
  1010,
  2501,
  3135,
  1998,
  3883,
  1012,
  2141,
  1998,
  2992,
  1999,
  5395,
  1010,
  3146,
  1010,
  2016,
  2864,
  1999,
  2536,
  4823,
  1998,
  5613,
  6479,
  2004,
  1037,
  2775,
  1010,
  1998,
  3123,
  2000,
  4476,
  1999,
  1996,
  2397,
  4134,
  2004,
  2599,
  3220,
  1997,
  1054,
  1004,
  1038,
  2611,
  1011,
  2177,
  10461,
  1005,
  1055,
  2775,
  1012,
  3266,
  2011,
  2014,
  2269,
  1010,
  25436,
  22815,
  1010,
  1996,
  2177,
  2150,
  2028,
  1997,
  1996,
  2088,
  1005,
  1055,
  2190,
  1011,
  4855,
  2611,
  2967,
  1997,
  2035,
  2051,
  1012,
  2037,
  142

In [81]:
small_val_set = tokenized_squad["validation"].select(range(1000))
small_train_set = tokenized_squad["train"].select(range(4000))

In [82]:
for name, param in qa_bert.bert.named_parameters ():
    if 'encoder.layer.23' in name:
        break
    param.requires_grad = False # disable training in BERT
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)

In [83]:
batch_size = 32
epochs = 2

In [84]:
import numpy as np

def compute_metrics(p):
    # Extract the start and end logits from the tuple of predictions
    start_logits, end_logits = p.predictions
    # The model's predicted start and end positions
    start_preds = np.argmax(start_logits, axis=1)
    end_preds = np.argmax(end_logits, axis=1)

    # If p.label_ids is a tuple, access the start and end positions accordingly
    start_positions = p.label_ids[0]  # Adjust this line based on your actual data structure
    end_positions = p.label_ids[1]    # Adjust this line as well

    # Compute Accuracy (number of correct predictions for both start and end positions)
    accuracy = np.mean((start_preds == start_positions) & (end_preds == end_positions))

    # Compute Exact Match (EM) metric
    exact_match = accuracy  # EM is same as accuracy in this case
    exact_match = exact_match.mean()  # Ensure exact_match is scalar

    # Compute F1 Score (based on start and end positions)
    def compute_f1(pred, true):
        overlap = 0
        for p, t in zip(pred, true):  # Iterate through each pair of predicted and true positions
            pred_span = [p, t]  # Predicted and true positions (start and end)
            # Calculate overlap between the prediction and the true span
            overlap += max(0, min(pred_span[1], t) - max(pred_span[0], p) + 1)  # Corrected overlap logic
        f1 = overlap / (len(pred) + len(true))  # Normalization for F1 score
        return f1

    f1 = compute_f1(start_preds, start_positions) + compute_f1(end_preds, end_positions)

    return {"accuracy": accuracy, "exact_match": exact_match, "f1": f1}

In [85]:
from sklearn.metrics import accuracy_score, f1_score as skl_f1
import numpy as np
from transformers import Trainer, TrainingArguments

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

training_arguments = TrainingArguments(
    output_dir='./qa/results',
    num_train_epochs=epochs,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    load_best_model_at_end=True,
    weight_decay=0.05,
    logging_steps=1,
    log_level="info",
    eval_strategy='epoch',
    save_strategy='epoch',
)

qa_bert.to(device)

trainer = Trainer(
    model=qa_bert,
    args=training_arguments,
    train_dataset=small_train_set,
    eval_dataset=small_val_set,
    tokenizer=bert_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# print(trainer.evaluate())

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/var/folders/tq/jtgfkbln5bv4lmlbq2tvnq1h0000gn/T/ipykernel_26254/1260625377.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [86]:
trainer.train()

***** Running training *****
  Num examples = 4,000
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 250
  Number of trainable parameters = 12,598,274
                                       
  0%|          | 0/250 [00:40<?, ?it/s]          

{'loss': 6.4081, 'grad_norm': 12.986937522888184, 'learning_rate': 4.9800000000000004e-05, 'epoch': 0.01}


                                       
  0%|          | 0/250 [00:56<?, ?it/s]          

{'loss': 6.2516, 'grad_norm': 14.245060920715332, 'learning_rate': 4.96e-05, 'epoch': 0.02}


                                       
  0%|          | 0/250 [01:13<?, ?it/s]          

...

Model weights saved in ./qa/results/checkpoint-250/model.safetensors
tokenizer config file saved in ./qa/results/checkpoint-250/tokenizer_config.json
Special tokens file saved in ./qa/results/checkpoint-250/special_tokens_map.json

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
































                                         

                                         
  0%|          | 0/250 [4:03:28<?, ?it/s]          

Saving model checkpoint to ./qa/results/checkpoint-250
Configuration saved in ./qa/results/checkpoint-250/config.json


{'eval_loss': 0.582841694355011, 'eval_accuracy': 0.901, 'eval_exact_match': 0.901, 'eval_f1': 1.5695000000000001, 'eval_runtime': 517.4789, 'eval_samples_per_second': 1.932, 'eval_steps_per_second': 0.062, 'epoch': 2.0}


Model weights saved in ./qa/results/checkpoint-250/model.safetensors
tokenizer config file saved in ./qa/results/checkpoint-250/tokenizer_config.json
Special tokens file saved in ./qa/results/checkpoint-250/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./qa/results/checkpoint-250 (score: 0.582841694355011).
                                         
100%|██████████| 250/250 [4:03:15<00:00, 58.38s/it]

{'train_runtime': 14595.9477, 'train_samples_per_second': 0.548, 'train_steps_per_second': 0.017, 'train_loss': 1.0444860431849956, 'epoch': 2.0}


TrainOutput(global_step=250, training_loss=1.0444860431849956, metrics={'train_runtime': 14595.9477, 'train_samples_per_second': 0.548, 'train_steps_per_second': 0.017, 'total_flos': 5572241952768000.0, 'train_loss': 1.0444860431849956, 'epoch': 2.0})

In [87]:
pipe = pipeline("question-answering", model=qa_bert, tokenizer=bert_tokenizer)
pipe('In what country is Normandy located?', 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/opt/homebrew/Caskroom/miniconda/base/envs/pytorch_env/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 1.6813191905384883e-07,
 'start': 21,
 'end': 30,
 'answer': 'Nourmands'}

In [88]:
squad['validation'][0]

{'id': '56ddde6b9a695914005b9628',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {'text': ['France', 'France', 'France', 'France'],
  'answer_start': [159, 159, 159, 159]}}

In [89]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
100%|██████████| 32/32 [10:49<00:00, 20.31s/it]


{'eval_loss': 0.582841694355011,
 'eval_accuracy': 0.901,
 'eval_exact_match': 0.901,
 'eval_f1': 1.5695000000000001,
 'eval_runtime': 674.8001,
 'eval_samples_per_second': 1.482,
 'eval_steps_per_second': 0.047,
 'epoch': 2.0}